# A03. `TRIPOLpy` Tutorial

Here I will reduce the 2019-05-03 TRIPOL observation.

First, download and install [``TRIPOLpy`` package](https://github.com/ysBach/TRIPOLpy) (**please read the README** there). 

Refer to the [TRIPOL lecture note](https://github.com/ysBach/AO2019/blob/master/Notebooks/A02_TRIPOL.md) for how to access the data.

Because there were some problems in the header of 2019-05-03 observation (``RET-ANG1``, which is the half-wave plate angle), you have to manually input the value. I chose this observation because it is rather different from usual TRIPOL data and you can experience how to overcome such problems within ``TRIPOLpy``.

The contents below follow this format:
```
1. The code line is given.
2. Simple explanation of the code line.
3. The snippet of the original source code (for the full source code, please see TRIPOLpy)
```

* **NOTE**: the ``self`` in the first arguments below are for the python ``class``. Please ignore it if you want to utilize the snippets I showed here to your ``function`` object.

## 1. Importing and Setting Up

In [1]:
from astropy.io import fits
from pathlib import Path
from tripolpy import preprocessor

TOPPATH = Path('.')
RAWPATH = TOPPATH / "rawdata"

## 2. Initiation

In [ ]:
p = preprocessor.Preprocessor(topdir=TOPPATH, rawdir=RAWPATH)

The above is identical to 

```python
USEFUL_KEYS = ["EXPTIME", "FILTER", "DATE-OBS", "RET-ANG1", "CCD_TEMP",
               "CCD_COOL", "OBJECT", "EPOCH", "RA", "DEC", "ALT", "AZ",
               "AIRMASS"]
p = preprocessor.Preprocessor(topdir=TOPPATH, rawdir=RAWPATH,
                              summary_keywords=USEFUL_KEYS)
```

because the original source code (you can verify by yourself from the source code) is like this:
```python
class Preprocessor():
    def __init__(self, topdir, rawdir, summary_keywords=USEFUL_KEYS):
        """
        Parameters
        ----------
        topdir : path-like
            The top directory of which all the other paths will be represented
            relative to.

        rawdir: path-like
            The directory where all the FITS files are stored (without any
            subdirectory)

        summary_keywords: list of str, optional
            The keywords of the header to be used for the summary table.
        """
        # code goes here
```

## 3. Organize the TRIPOL images.

In [ ]:
p.organize_tripol(archive_dir=RAWPATH / "archive")

This first **updates header** appropriately, and then **organizes** the files based on the header. It automatically finds bias, dark, flat, and object frames. For all such image types, it separates them first by filter and then by object name (``OBJECT`` key of the header).

This is identical to use the following code:
```python
rename_by = ["COUNTER", "FILTER", "OBJECT", "EXPOS", "RET-ANG1"]
mkdir_by = ["FILTER", "OBJECT"]
p.organize_tripol(rename_by=rename_by,
                  mkdir_by=mkdir_by, 
                  delimiter='_',
                  archive_dir=RAWPATH / "archive")
```

This is because, by default, the ``.organize_tripol()`` method is defined as this:

```python
def organize_tripol(self,
                    rename_by=["COUNTER", "FILTER",
                               "OBJECT", "EXPOS", "RET-ANG1"],
                    mkdir_by=["FILTER", "OBJECT"], delimiter='_',
                    archive_dir=None, verbose=False):
    ''' Rename FITS files after updating theur headers.
        Parameters
        ----------
        fpath: path-like
            The path to the target FITS file.
        rename_by: list of str
            The keywords in header to be used for the renaming of FITS files.
            Each keyword values are connected by ``delimiter``.
        mkdir_by: list of str, optional
            The keys which will be used to make subdirectories to classify
            files. If given, subdirectories will be made with the header value
            of the keys.
        delimiter: str, optional
            The delimiter for the renaming.
        archive_dir: path-like or None, optional
            Where to move the original FITS file. If ``None``, the original file
            will remain there. Deleting original FITS is dangerous so it is only
            supported to move the files. You may delete files manually if
            needed.
    '''
    # code goes here
```

* **TIP**: Now see the raw data directory. You can see the data are re-arranged into many separate folders and now have meaningful file names. Also many folders will have been generated by now.

## 4. Bias Combine (Master Bias or Master Zero)

In [ ]:
p.make_bias(savedir=TOPPATH / "calibration")

This extracts only the bias frames from the previously organized files and do bias combine. Then the master bias is saved to ``savedir``. 

This is identical to the following code:
``` python
MEDCOMB_KEYS = dict(overwrite=True,
                    unit='adu',
                    combine_method="median",
                    reject_method=None,
                    combine_uncertainty_function=None)

p.make_bias(savedir=TOPPATH / "calibration",
            hdr_keys="OBJECT", 
            hdr_vals="bias",
            group_by=["FILTER"], 
            delimiter='_', 
            dtype='float32',
            comb_kwargs=MEDCOMB_KEYS
            )
```

Since the related source codes are too long, I only included core parts. If you are interested in, please check the original source code by yourself.


```python
def make_bias(self, savedir=None, hdr_keys="OBJECT", hdr_vals="bias",
              group_by=["FILTER"], delimiter='_', dtype='float32',
              comb_kwargs=MEDCOMB_KEYS):
    ''' Finds and make bias frames.
    Parameters
    ----------
    savedir : path-like, optional.
        The directory where the frames will be saved.

    hdr_key : str or list of str, optional
        The header keys to be used for the identification of the bias
        frames. Each value should correspond to the same-index element of
        ``hdr_val``.

    hdr_val : str, float, int or list of such, optional
        The header key and values to identify the bias frames. Each value
        should correspond to the same-index element of ``hdr_key``.

    group_by : None, str or list str, optional.
        The header keywords to be used for grouping frames. For dark
        frames, usual choice can be ``['EXPTIME']``.

    delimiter : str, optional.
        The delimiter for the renaming.

    dtype : str or numpy.dtype object, optional.
        The data type you want for the final master bias frame. It is 
        recommended to use ``float32`` or ``int16`` if there is no 
        specific reason.

    comb_kwargs: dict or None, optional.
        The parameters for ``combine_ccd``.
    '''
    # code goes here

```

```python
def combine_ccd(fitslist, trim_fits_section=None, output=None, unit='adu',
                subtract_frame=None, combine_method='median', reject_method=None,
                normalize_exposure=False, exposure_key='EXPTIME',
                combine_uncertainty_function=ccdproc_mad2sigma_func,
                extension=0, dtype=np.float32, type_key=None, type_val=None,
                output_verify='fix', overwrite=False,
                **kwargs):
    ''' Combining images
    # See original source code for the details.
    '''
    # code goes here
```

## 5. Dark Combine (Master Dark for Each Exposure Time)

In [ ]:
p.make_dark(savedir=TOPPATH / "calibration")

This code makes dark frames. The dark images are grouped by filter and exposure time.

This is identical to the following code:
```python
MEDCOMB_KEYS = dict(overwrite=True,
                    unit='adu',
                    combine_method="median",
                    reject_method=None,
                    combine_uncertainty_function=None)

p.make_dark(savedir=TOPPATH / "calibration",
            hdr_keys="OBJECT", 
            hdr_vals="dark",
            bias_sub=True,
            group_by=["FILTER", "EXPTIME"], 
            bias_grouped_by=["FILTER"],
            exposure_key="EXPTIME", 
            dtype='float32',
            delimiter='_', 
            comb_kwargs=MEDCOMB_KEYS)
```

The original method source code is like this: 

```python
def make_dark(self, savedir=None, hdr_keys="OBJECT", hdr_vals="dark",
              bias_sub=True,
              group_by=["FILTER", "EXPTIME"], bias_grouped_by=["FILTER"],
              exposure_key="EXPTIME", dtype='float32',
              delimiter='_', comb_kwargs=MEDCOMB_KEYS):
    """ Makes and saves dark (bias subtracted) images.
    Parameters
    ----------
    savedir: path-like, optional
        The directory where the frames will be saved.

    hdr_key : str or list of str, optional
        The header keys to be used for the identification of the bias
        frames. Each value should correspond to the same-index element of
        ``hdr_val``.

    hdr_val : str, float, int or list of such, optional
        The header key and values to identify the bias frames. Each value
        should correspond to the same-index element of ``hdr_key``.

    bias_sub: bool, optional
        If ``True``, subtracts bias from dark frames using self.biaspahts.

    group_by: None, str or list str, optional
        The header keywords to be used for grouping frames. For dark
        frames, usual choice can be ``['EXPTIME']``.

    bias_grouped_by: str or list of str, optional
        How the bias frames are grouped by.

    exposure_key: str, optional
        If you want to make bias from a list of dark frames, you need to
        let the function know the exposure time of the frames, so that the
        miniimum exposure time frame will be used as bias. Default is
        "EXPTIME".

    comb_kwargs: dict or None, optional
        The parameters for ``combine_ccd``.
    """
    # Code goes here

```

## 6. Flat Combine (Master Flat)

In [ ]:
p.make_flat(savedir=TOPPATH / "calibration", group_by=["FILTER"])

This makes flat images. First, it selects flat images for each filter. Then it again group by the half-wave plate angles. 

* **NOTE**: In 2019-05-03 observation, we took only half-wave plate angle of 0 degree. To use this flat throughout all the half-wave plate angles (which is not desired but the best way we can do given the data set), we forced the code **not** to distinguish flats based on the angle (header keyword is ``RET-ANG1``).

The above is identical to the following code:
```python
MEDCOMB_KEYS = dict(overwrite=True,
                    unit='adu',
                    combine_method="median",
                    reject_method=None,
                    combine_uncertainty_function=None)

p.make_flat(savedir=TOPPATH / "calibration", group_by=["FILTER"])
```

The original source code looks like below. As you can see, if you use ``.make_flat()`` without specifying ``group_by``, it will by default use ``group_by=["FILTER", "RET-ANG1"]``. 

```python
def make_flat(self, savedir=None,
              hdr_keys=["OBJECT"], hdr_vals=["flat"],
              group_by=["FILTER", "RET-ANG1"],
              bias_sub=True, dark_sub=True,
              bias_grouped_by=["FILTER"],
              dark_grouped_by=["FILTER", "EXPTIME"],
              exposure_key="EXPTIME",
              comb_kwargs=MEDCOMB_KEYS, delimiter='_', dtype='float32'):
    '''Makes and saves flat images.
    Parameters
    ----------
    savedir: path-like, optional
        The directory where the frames will be saved.

    hdr_key : str or list of str, optional
        The header keys to be used for the identification of the bias
        frames. Each value should correspond to the same-index element of
        ``hdr_val``.

    hdr_val : str, float, int or list of such, optional
        The header key and values to identify the bias frames. Each value
        should correspond to the same-index element of ``hdr_key``.

    bias_sub, dark_sub : bool, optional
        If ``True``, subtracts bias and dark frames using ``self.biaspahts``
        and ``self.darkpaths``.

    group_by: None, str or list str, optional
        The header keywords to be used for grouping frames. For dark
        frames, usual choice can be ``['EXPTIME']``.

    bias_grouped_by, dark_grouped_by : str or list of str, optional
        How the bias and dark frames are grouped by.

    exposure_key: str, optional
        If you want to make bias from a list of dark frames, you need to
        let the function know the exposure time of the frames, so that the
        miniimum exposure time frame will be used as bias. Default is
        "EXPTIME".

    comb_kwargs: dict or None, optional
        The parameters for ``combine_ccd``.
    '''
    # Code goes here
```

## 7. Preprocess at Once

In [ ]:
summary_reduced = p.do_preproc(savedir=TOPPATH / "processed",
                              flat_grouped_by=["FILTER"])

It now does the preprocessing and summarize the processed images as ``astropy Table`` object. 

It is identical to use
```python
summary_reduced = p.do_preproc(savedir=TOPPATH / "processed",
                               delimiter='_', 
                               dtype='float32',
                               bias_grouped_by=["FILTER"],
                               dark_grouped_by=["FILTER", "EXPTIME"],
                               flat_grouped_by=["FILTER"],
                               verbose_bdf=True, 
                               verbose_summary=False)
```

Note that, the original source code (see below), ``flat_grouped_by=["FILTER", "RET-ANG1"]`` by default. As I did **NOT** use ``"RET-ANG1"`` (the half-wave plate angle) for the flat grouping, I had to use ``flat_grouped_by=["FILTER"]``.

```python
def do_preproc(self, savedir=None, delimiter='_', dtype='float32',
               bias_grouped_by=["FILTER"],
               dark_grouped_by=["FILTER", "EXPTIME"],
               flat_grouped_by=["FILTER", "RET-ANG1"],
               verbose_bdf=True, verbose_summary=False):
    ''' Conduct the preprocessing using simplified ``bdf_process``.
    Parameters
    ----------
    savedir: path-like, optional
        The directory where the frames will be saved.

    delimiter : str, optional.
        The delimiter for the renaming.

    dtype : str or numpy.dtype object, optional.
        The data type you want for the final master bias frame. It is
        recommended to use ``float32`` or ``int16`` if there is no
        specific reason.

    bias_grouped_by, dark_grouped_by : str or list of str, optional
        How the bias, dark, and flat frames are grouped by.
    '''
    # Code goes here
```


## 8. View the Summary
Finally, print out the results:

In [ ]:
summary_reduced

* **TIP**: You can view the resulting summary file with, e.g., Excel, by ``[your_RAWPATH]/summary_reduced.csv``.